Skew diagram - difference between G and C. On forward (lagging) - G is high and C is low. On reverse (leading) - C is high and G is low

In [ ]:
import numpy as np

def SkewDiagram(genome):
    """
    Computes the skew diagram of a genome sequence.  
    Skew is defined as the cumulative difference between the counts of 'G' and 'C' nucleotides.
    The function returns the positions (indices) where the skew is minimal.

    Args:
        genome (str): DNA sequence (genome).

    Returns:
        list: List of indices where the skew reaches its minimum value.
    """
    skew = [0] * (len(genome) + 1)  # Initialize skew list with 0 at position 0

    for i in range(len(genome)):
        if genome[i] == 'C':
            skew[i + 1] = skew[i] - 1  # Decrement skew for 'C'
        elif genome[i] == 'G':
            skew[i + 1] = skew[i] + 1  # Increment skew for 'G'
        else:
            skew[i + 1] = skew[i]      # No change for other nucleotides

    # Find indices where skew reaches its minimum value
    min_skew = np.where(np.array(skew) == np.min(skew))
    return min_skew[0].tolist()


def read_genome_from_file(file_path):
    """
    Reads a genome sequence from a FASTA file, ignoring header lines.

    Args:
        file_path (str): Path to the FASTA file.

    Returns:
        str: Concatenated genome sequence.
    """
    genome = ''
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('>'):  # Skip header lines starting with '>'
                genome += line.strip()    # Remove newline and append
    return genome


# Example usage
SkewDiagram('GCATACACTTCCCAGTAGGTACTG')


[13, 14]

In [2]:
def HammingDistance(seq1, seq2):
    count = 0
    for n,j in zip(seq1, seq2):
        if n != j:
            count += 1
        else:
            pass
    return count

seq1 = 'CTTGAAGTGGACCTCTAGTTCCTCTACAAAGAACAGGTTGACCTGTCGCGAAG'
seq2 = 'ATGCCTTACCTAGATGCAATGACGGACGTATTCCTTTTGCCTCAACGGCTCCT'

HammingDistance(seq1, seq2)

43

Approximate Pattern Matching Problem

In [3]:
def ApproximateMatch(pattern, text, k):
    indices = []
    for i in range(len(text) - len(pattern) + 1):
        if HammingDistance(pattern, text[i:i+len(pattern)]) <= k:
            indices.append(i)

    return len(indices)

pattern = 'TGT'
k = 1
text = 'CGTGACAGTGTATGGGCATCTTT'
ApproximateMatch(pattern, text, k)

8

Most Frequent k-mer with up to d mismatches

In [1]:
def ApproximateMatch(pattern, text, d):
    """
    Finds all starting indices where 'pattern' appears in 'text' with at most 'd' mismatches.

    Args:
        pattern (str): Pattern to search for.
        text (str): Text to search within.
        d (int): Maximum number of mismatches allowed.

    Returns:
        list: List of starting indices where the pattern approximately matches the text.
    """
    indices = []
    for i in range(len(text) - len(pattern) + 1):
        if HammingDistance(pattern, text[i:i+len(pattern)]) <= d:
            indices.append(i)
    return indices


def Neighbors(pattern, d):
    """
    Recursively generates the d-neighborhood of a pattern (i.e. all strings within d mismatches).

    Args:
        pattern (str): DNA string.
        d (int): Maximum number of mismatches.

    Returns:
        set: Set of strings within d mismatches of the pattern.
    """
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return {'A', 'C', 'G', 'T'}
    
    alphabet = {'A', 'C', 'G', 'T'}
    neighbors = set()
    suffix_neighbors = Neighbors(pattern[1:], d)

    for text in suffix_neighbors:
        # If suffix is within d-1, we can afford to mutate first character
        if HammingDistance(pattern[1:], text) < d:
            for nucleotide in alphabet:
                neighbors.add(nucleotide + text)
        else:
            neighbors.add(pattern[0] + text)
    return neighbors


def ImmediateNeighbors(pattern):
    """
    Returns all patterns that differ from the input pattern by **exactly one** character.

    Args:
        pattern (str): DNA string.

    Returns:
        set: Set of strings differing by one character.
    """
    neighbors = set()
    alphabet = {'A', 'C', 'G', 'T'}
    for i in range(len(pattern)):
        for nucleotide in alphabet:
            if nucleotide != pattern[i]:
                neighbors.add(pattern[:i] + nucleotide + pattern[i+1:])
    return neighbors


def IterativeNeighbors(pattern, d):
    """
    Iteratively builds the d-neighborhood of a pattern using immediate neighbors.

    Args:
        pattern (str): DNA pattern.
        d (int): Maximum Hamming distance.

    Returns:
        set: All patterns within d mismatches of input pattern.
    """
    neighborhood = {pattern}
    for j in range(d):
        new_neighbors = set()
        for pattern_prime in neighborhood:
            new_neighbors.update(ImmediateNeighbors(pattern_prime))
        neighborhood.update(new_neighbors)
    return neighborhood


def MostFrequentKmer(text, d, k):
    """
    Finds the most frequent k-mers (and their reverse complements) with at most d mismatches.

    Args:
        text (str): Input DNA sequence.
        d (int): Allowed mismatches.
        k (int): Length of k-mer.

    Returns:
        str: Space-separated string of most frequent k-mers.
    """
    freqmap = {}
    patterns = []

    for j in range(len(text) - k + 1):
        pattern = text[j:j+k]
        pattern_rc = sequencecomplement(pattern)

        # Neighborhoods for both pattern and its reverse complement
        neighborhood = Neighbors(pattern, d)
        neighborhood_rc = Neighbors(pattern_rc, d)

        for neighbor in neighborhood:
            freqmap[neighbor] = freqmap.get(neighbor, 0) + 1

        for neighbor in neighborhood_rc:
            rc_neighbor = sequencecomplement(neighbor)
            freqmap[rc_neighbor] = freqmap.get(rc_neighbor, 0) + 1

    # Find max frequency
    max_freq = max(freqmap.values())
    for pattern in freqmap:
        if freqmap[pattern] == max_freq:
            patterns.append(pattern)

    return ' '.join(map(str, patterns))


def sequencecomplement(seq):
    """
    Computes the reverse complement of a DNA sequence.

    Args:
        seq (str): DNA string.

    Returns:
        str: Reverse complement of the input sequence.
    """
    return complement(reverse(seq))


def reverse(seq):
    """
    Reverses a DNA string.

    Args:
        seq (str): DNA string.

    Returns:
        str: Reversed string.
    """
    return seq[::-1]


def complement(seq):
    """
    Returns the complement of a DNA string.

    Args:
        seq (str): DNA string.

    Returns:
        str: Complemented string.
    """
    basecomplement = {
        'A':'T', 'T':'A', 'C':'G', 'G':'C',
        'a':'t', 't':'a', 'c':'g', 'g':'c',
        'N':'N', 'n':'n'
    }
    letters = [basecomplement[base] for base in seq]
    return ''.join(letters)


# Example usage:
text = 'CCAGTCAATG'
d = 1
k = 10

# Print number of neighbors within Hamming distance d
print(len(IterativeNeighbors(text, d)))


31


In [ ]:
def MostFrequentKmer(text, d, k):
    """
    Finds the most frequent k-mers (and their reverse complements) in a DNA sequence
    allowing up to d mismatches.

    Args:
        text (str): DNA sequence.
        d (int): Maximum number of mismatches.
        k (int): Length of k-mers.

    Returns:
        str: Space-separated string of most frequent k-mers.
    """
    freqmap = {}
    patterns = []

    for j in range(len(text) - k + 1):
        pattern = text[j:j+k]
        pattern_rc = sequencecomplement(pattern)

        # Generate all k-mers within d mismatches (neighborhood) of pattern and its reverse complement
        neighborhood = Neighbors(pattern, d)
        neighborhood_rc = Neighbors(pattern_rc, d)

        for neighbor in neighborhood:
            freqmap[neighbor] = freqmap.get(neighbor, 0) + 1

        for neighbor_rc in neighborhood_rc:
            freqmap[neighbor_rc] = freqmap.get(neighbor_rc, 0) + 1

    # Find the maximum frequency
    m = max(freqmap.values())

    # Collect all patterns that occur with maximum frequency
    for pattern in freqmap:
        if freqmap[pattern] == m:
            patterns.append(pattern)

    return ' '.join(map(str, patterns))


def MismatchedClump(text, d, k, L, t):
    """
    Finds all k-mers forming (L, t, d)-clumps in a DNA sequence. A (L, t, d)-clump is a k-mer that appears
    at least t times (with up to d mismatches) within a window of length L.

    Args:
        text (str): Input DNA sequence.
        d (int): Maximum number of mismatches.
        k (int): Length of k-mers.
        L (int): Window length.
        t (int): Minimum number of (approximate) occurrences in the window.

    Returns:
        list: List of distinct k-mers forming (L, t, d)-clumps.
    """
    patterns = set()  # Store resulting patterns
    freqmap = {}

    for i in range(len(text) - L + 1):
        window = text[i:i+L]
        freqmap = {}

        # Count all approximate k-mers (and their reverse complements) in the window
        for j in range(len(window) - k + 1):
            pattern = window[j:j+k]
            pattern_rc = sequencecomplement(pattern)

            neighborhood = Neighbors(pattern, d)
            neighborhood_rc = Neighbors(pattern_rc, d)

            for neighbor in neighborhood:
                freqmap[neighbor] = freqmap.get(neighbor, 0) + 1

            for neighbor_rc in neighborhood_rc:
                rc = sequencecomplement(neighbor_rc)
                freqmap[rc] = freqmap.get(rc, 0) + 1

        # Add patterns that meet the threshold
        for key, value in freqmap.items():
            if value >= t:
                patterns.add(key)

    return list(patterns)


In [35]:
text = read_genome_from_file('Salmonella_enterica.txt')
min_indices = SkewDiagram(text)
DnaA = MismatchedClump(text[min_indices[0]-500:min_indices[0]+500], 1, 9, 250, 4)
print(DnaA)

['ACTTCAGCA', 'AGCGTCTTT', 'AATTCAGCC', 'AGCTCGTAC', 'GCAGATCCC', 'ATCATCAAG', 'CGCTACAGC', 'AGTACACTG', 'GTCCACAGA', 'AACCAATTG', 'CAATTGATG', 'ACAAGTGAT', 'GCCTTTTTA', 'AAATTGATG', 'AACAATTGA', 'TACAGCCTC', 'GTAGAACCC', 'CGATCAAGC', 'AGCTTCTTT', 'CAATACAGC', 'AATTGATGT', 'CAAGTGATG', 'CCAGCACGT', 'ACTACACCA', 'CATCTTTAC', 'AATTGACGA', 'TTCACTGAG', 'CAATTGAAG', 'TCAATTGAT', 'CCACAGGTC', 'ATCCCACCT', 'CAATTGAGG', 'AGTCCAATG', 'AATGCAGTC', 'CCACCGGTA', 'CACTGCAGC', 'CTTTATTAA', 'CAGCTCGTA', 'AAGCAATTG', 'GCCTCTTTA', 'CTTCAGCAT', 'GCAGCATCT', 'ATGATCGTC', 'ATCGGCTTG', 'AAAACAATT', 'AAGTGATGA', 'TCAGCTCGT', 'CACTTCAGC', 'ACCACAGGT', 'AGCCTCTTT', 'CCTCTTTAT', 'GATCGCCTT', 'TCCACAGAG', 'TTGATAAAA', 'TTGATCCCA', 'GATCGTCTT', 'TCATGTGCA', 'ACAATTGAC', 'TTTTCAACA', 'GATCGACTT', 'AGATCCCAA', 'AAAATTGAT', 'GTTCACTGA', 'CAGCATCTT', 'GCATTTTTA', 'ATCCCAGAT', 'AATTGATGC', 'AGCATCTTT', 'CAATTGACG', 'TACACCATC', 'AGCAATTGA', 'AACCAGATC', 'TGATCGCCT', 'AAGATCCCA', 'CGTCTTACA', 'CAGCACGTT', 'CAATGATCG'

In [43]:
text = read_genome_from_file('Salmonella_enterica.txt')
min_indices = SkewDiagram(text)
DnaA = MismatchedClump(text[min_indices[0]-500:min_indices[0]+500], 1, 9, 500, 6)
print(DnaA)

[]


In [40]:
print(len('aatgatgatgacgtcaaaaggatccggataaaacatggtgattgcctcgcataacgcggtatgaaaatggattgaagcccgggccgtggattctactcaactttgtcggcttgagaaagacctgggatcctgggtattaaaaagaagatctatttatttagagatctgttctattgtgatctcttattaggatcgcactgcccTGTGGATAAcaaggatccggcttttaagatcaacaacctggaaaggatcattaactgtgaatgatcggtgatcctggaccgtataagctgggatcagaatgaggggTTATACACAactcaaaaactgaacaacagttgttcTTTGGATAActaccggttgatccaagcttcctgacagagTTATCCACAgtagatcgcacgatctgtatacttatttgagtaaattaacccacgatcccagccattcttctgccggatcttccggaatgtcgtgatcaagaatgttgatcttcagtg'))

500


In [34]:
text = read_genome_from_file('Salmonella_enterica.txt')
min_indices = SkewDiagram(text)
DnaA = MismatchedClump(text[min_indices[0]-500:min_indices[0]+500], 1, 9, 250, 5)
print(DnaA)


['GCTTCTTTA']


In [32]:
print(len(DnaA))

120
